In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!gunzip -k /content/drive/MyDrive/cancerData/camelyonpatch_level_2_split_test_x.h5.gz
!gunzip -k /content/drive/MyDrive/cancerData/camelyonpatch_level_2_split_test_y.h5.gz
!gunzip -k /content/drive/MyDrive/cancerData/camelyonpatch_level_2_split_valid_x.h5.gz
!gunzip -k /content/drive/MyDrive/cancerData/camelyonpatch_level_2_split_valid_y.h5.gz


gzip: /content/drive/MyDrive/cancerData/camelyonpatch_level_2_split_test_x.h5 already exists; do you wish to overwrite (y or n)? n
	not overwritten
gzip: /content/drive/MyDrive/cancerData/camelyonpatch_level_2_split_test_y.h5 already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [13]:
import os
os.listdir('/content/drive/MyDrive/cancerData')


['camelyonpatch_level_2_split_train_x.h5.gz',
 'camelyonpatch_level_2_split_train_y.h5.gz',
 'camelyonpatch_level_2_split_valid_y.h5.gz',
 'camelyonpatch_level_2_split_valid_x.h5.gz',
 'camelyonpatch_level_2_split_test_x.h5.gz',
 'camelyonpatch_level_2_split_test_y.h5.gz',
 'camelyonpatch_level_2_split_test_y.h5',
 'camelyonpatch_level_2_split_test_x.h5',
 'camelyonpatch_level_2_split_valid_x.h5',
 'camelyonpatch_level_2_split_valid_y.h5']

In [135]:
import h5py
import matplotlib.pyplot as plt

# مسیرها
path_x_test = '/content/drive/MyDrive/cancerData/camelyonpatch_level_2_split_test_x.h5'
path_y_test = '/content/drive/MyDrive/cancerData/camelyonpatch_level_2_split_test_y.h5'
path_x_val = '/content/drive/MyDrive/cancerData/camelyonpatch_level_2_split_valid_x.h5'
path_y_val = '/content/drive/MyDrive/cancerData/camelyonpatch_level_2_split_valid_y.h5'

# باز کردن فایل‌ها
with h5py.File(path_x_test, 'r') as fx, h5py.File(path_y_test, 'r') as fy:
    print("📁 کلیدهای داخل X:", list(fx.keys()))
    print("📁 کلیدهای داخل Y:", list(fy.keys()))

    train_im = fx['x'][:2000]
    train_lable = fy['y'][:2000]

with h5py.File(path_x_val, 'r') as fx, h5py.File(path_y_val, 'r') as fy:
    print("📁 کلیدهای داخل X:", list(fx.keys()))
    print("📁 کلیدهای داخل Y:", list(fy.keys()))

    test_im = fx['x'][:20000]
    test_lable = fy['y'][:20000]

print("شکل X:", train_im.shape)
print("شکل Y:", train_lable.shape)
print("شکل X:", test_im.shape)
print("شکل Y:", test_lable.shape)



📁 کلیدهای داخل X: ['x']
📁 کلیدهای داخل Y: ['y']
📁 کلیدهای داخل X: ['x']
📁 کلیدهای داخل Y: ['y']
شکل X: (2000, 96, 96, 3)
شکل Y: (2000, 1, 1, 1)
شکل X: (20000, 96, 96, 3)
شکل Y: (20000, 1, 1, 1)


In [136]:
train_im[254].shape

(96, 96, 3)

In [137]:
train_lable[0]

array([[[0]]], dtype=uint8)

In [138]:
# قبل از ساخت Dataset
train_lable = train_lable.squeeze()  # (1000, 1, 1, 1) -> (1000,)
val_labele = test_lable.squeeze()      # (N,)


In [139]:
import numpy as np
from PIL import Image
from copy import deepcopy
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision.transforms.v2 import Compose, ToImage, Normalize, ToPILImage, Resize, ToDtype
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, MultiStepLR, CyclicLR, LambdaLR
import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [140]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, X, Y, image_size=(128,128)):
        """
        X: numpy array تصاویر، شکل N H W C
        Y: numpy array برچسب‌ها، شکل N
        image_size: tuple (H, W) برای resize
        """
        self.X = X
        self.Y = Y
        self.image_size = image_size

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]               # numpy
        label = self.Y[idx]

        # تبدیل numpy به PIL Image برای resize
        image = Image.fromarray(image.astype('uint8'))
        image = image.resize(self.image_size)

        # تبدیل به numpy float32 و scale بین 0 و 1
        image = np.array(image).astype(np.float32) / 255.0

        # تبدیل به tensor و HWC -> CHW
        image = torch.from_numpy(image).permute(2,0,1)

        # تبدیل label به tensor
        label = torch.tensor(label, dtype=torch.long)

        return image, label


In [141]:
train_dataset = CustomDataset(train_im , train_lable)
test_dataset = CustomDataset(test_im , test_lable)

train_loader = DataLoader(train_dataset , batch_size=32 , shuffle=True)
test_loader = DataLoader(test_dataset , batch_size=32 , shuffle=False)

In [142]:
def statistics_per_channel(images, labels):
    # NCHW
    n_samples, n_channels, n_height, n_weight = images.size()
    # Flatten HW into a single dimension
    flatten_per_channel = images.reshape(n_samples, n_channels, -1)
    # Computes statistics of each image per channel
    # Average pixel value per channel
    # (n_samples, n_channels)
    means = flatten_per_channel.mean(axis=2)
    # Standard deviation of pixel values per channel
    # (n_samples, n_channels)
    stds = flatten_per_channel.std(axis=2)
    # Adds up statistics of all images in a mini-batch
    # (1, n_channels)
    sum_means = means.sum(axis=0)
    sum_stds = stds.sum(axis=0)
    # Makes a tensor of shape (1, n_channels)
    # with the number of samples in the mini-batch
    n_samples = torch.tensor([n_samples]*n_channels).float()
    # Stack the three tensors on top of one another
    # (3, n_channels)
    return torch.stack([n_samples, sum_means, sum_stds], axis=0)

In [143]:
results = [statistics_per_channel(x, y) for i, (x, y) in enumerate(train_loader)]
results = torch.stack(results, axis=0)
results = results.sum(axis=0)
total_samples, total_means, total_stds = results
norm_mean = total_means / total_samples
norm_std = total_stds / total_samples
norm_mean,norm_std

(tensor([0.6843, 0.5275, 0.6953]), tensor([0.1766, 0.1955, 0.1535]))

In [144]:
# قبل از ساخت Dataset
train_lable = train_lable.squeeze()  # (1000, 1, 1, 1) -> (1000,)
test_labele = test_lable.squeeze()      # (N,)

In [145]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, X, Y, image_size=(128,128) , mean = (0.6843, 0.5275, 0.6953) , std = (0.1766, 0.1955, 0.1535)):
        """
        X: numpy array تصاویر، شکل N H W C
        Y: numpy array برچسب‌ها، شکل N
        image_size: tuple (H, W) برای resize
        """
        self.X = X
        self.Y = Y
        self.image_size = image_size
        self.mean = mean
        self.std = std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]               # numpy
        label = self.Y[idx]

        # تبدیل numpy به PIL Image برای resize
        image = Image.fromarray(image.astype('uint8'))
        image = image.resize(self.image_size)

        # تبدیل به numpy float32 و scale بین 0 و 1
        image = np.array(image).astype(np.float32) / 255.0

        # نورمالیزیشن کانال به کانال
        image = (image - np.array(self.mean)) / np.array(self.std)

        # تبدیل به tensor و HWC -> CHW
        image = torch.from_numpy(image).permute(2,0,1).float()

        # تبدیل label به tensor
        label = torch.tensor(label.squeeze(), dtype=torch.float).view(1)


        return image, label


In [146]:
train_dataset = CustomDataset(train_im , train_lable)
test_dataset = CustomDataset(test_im , test_lable)

train_loader = DataLoader(train_dataset , batch_size=32 , shuffle=True)
test_loader = DataLoader(test_dataset , batch_size=32 , shuffle=False)

In [147]:
train_dataset[250]

(tensor([[[ 0.5219,  1.0104,  0.7440,  ...,  1.4990,  1.2991,  1.0993],
          [-0.4329, -0.2553, -0.0110,  ...,  1.1881,  1.2991,  1.1881],
          [-0.8326, -0.7660, -0.5218,  ...,  0.7218,  1.0771,  1.4546],
          ...,
          [ 0.7884,  0.8772,  1.1659,  ...,  0.7440,  0.2777, -0.0998],
          [ 1.0549,  1.1215,  1.2991,  ...,  0.1666, -0.0554, -0.1443],
          [ 1.3879,  1.5212,  1.5434,  ...,  0.1444,  0.1888,  0.3443]],
 
         [[ 0.3508,  0.7520,  0.4310,  ...,  1.5744,  1.2735,  1.0729],
          [-0.5318, -0.3914, -0.2510,  ...,  1.3939,  1.4340,  1.2936],
          [-0.9129, -0.8728, -0.7124,  ...,  1.1532,  1.4340,  1.6947],
          ...,
          [ 0.0298,  0.1903,  0.5514,  ...,  0.5313, -0.1507, -0.6321],
          [ 0.1903,  0.3307,  0.6116,  ...,  0.0499, -0.3112, -0.5118],
          [ 0.4511,  0.6517,  0.8121,  ...,  0.0900, -0.0103,  0.0499]],
 
         [[ 0.2733,  0.7587,  0.3500,  ...,  1.3719,  1.1419,  0.8865],
          [-0.8763, -0.7230,

In [148]:
class CNN2(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3 , 20 , kernel_size=3)
    self.maxpool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(20, 30 , kernel_size=3)
    self.maxpool2 = nn.MaxPool2d(2)
    self.conv3 = nn.Conv2d(30, 50 , kernel_size=3)
    self.maxpool3 = nn.MaxPool2d(2)

    self.fc1 = nn.Linear(50 * 14 * 14 , 512)
    self.bn1 = nn.BatchNorm1d(512)
    self.fc2 = nn.Linear(512 , 256)
    self.bn2 = nn.BatchNorm1d(256)
    self.fc3 = nn.Linear(256 , 128)
    self.bn3 = nn.BatchNorm1d(128)
    self.fc4 = nn.Linear(128 , 1)

    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=.5)


  def featurizer(self, x):
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.relu(x)
    x = self.conv3(x)
    x = self.maxpool3(x)
    x = self.relu(x)
    x = torch.flatten(x , 1)
    return x

  def classifier(self, x):
    x = self.dropout(self.bn1(self.fc1(x)))
    x = self.relu(x)
    x = self.dropout(self.bn2(self.fc2(x)))
    x = self.relu(x)
    x = self.dropout(self.bn3(self.fc3(x)))
    x = self.relu(x)
    x = self.fc4(x)
    return x

  def forward(self , x):
    x = self.featurizer(x)
    x = self.classifier(x)
    return x


In [149]:
torch.manual_seed(42)
model = CNN2().to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters() , lr=0.01)

In [154]:
def binary_accuracy(outputs, labels):
    """محاسبه accuracy برای binary classification"""
    probs = torch.sigmoid(outputs)
    preds = (probs > 0.5).float()
    correct = (preds == labels).float().sum()
    return correct / labels.size(0)


def train(model, dataloader, optimizer,criterion):
    model.train()
    total_loss = 0.0
    total_acc = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_acc += accuracy(outputs, targets)
    return total_loss / len(dataloader), total_acc / len(dataloader)

def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            total_acc += accuracy(outputs, targets)
    return total_loss / len(dataloader), total_acc / len(dataloader)

n_epochs = 10
train_losses, test_losses = [], []
train_accuracies, test_accuracies = [], []
for epoch in range(n_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, loss_fn)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    test_loss, test_acc = validate(model, test_loader, loss_fn)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

    print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {test_loss:.4f}, "
          f"Train Accuracy: {train_acc:.4f}, Validation Accuracy: {test_acc:.4f}")

# Plot losses
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, n_epochs + 1), train_losses, label='Train Loss')
plt.plot(range(1, n_epochs + 1), test_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()

# Plot accuracies
plt.subplot(1, 2, 2)
plt.plot(range(1, n_epochs + 1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, n_epochs + 1), test_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracies')
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in  plt.gca().get_yticks()])
plt.legend()

plt.show()

Epoch 1/10, Train Loss: 0.0309, Validation Loss: 1.2768, Train Accuracy: 16.1746, Validation Accuracy: 16.0272
Epoch 2/10, Train Loss: 0.0254, Validation Loss: 1.1005, Train Accuracy: 16.1746, Validation Accuracy: 16.0272


KeyboardInterrupt: 

In [153]:
import torch
from torch import nn, optim
from tqdm import tqdm  # برای progress bar

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# مدل، optimizer و loss
model = CNN2().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCEWithLogitsLoss()

def binary_accuracy(outputs, labels):
    """محاسبه accuracy برای binary classification"""
    probs = torch.sigmoid(outputs)
    preds = (probs > 0.5).float()
    correct = (preds == labels).float().sum()
    return correct / labels.size(0)

def train_one_epoch(model, dataloader, optimizer, loss_fn):
    model.train()
    running_loss, running_acc = 0.0, 0.0
    for images, labels in tqdm(dataloader, leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        running_acc += binary_accuracy(outputs, labels) * images.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_acc / len(dataloader.dataset)
    return epoch_loss, epoch_acc

def validate(model, dataloader, loss_fn):
    model.eval()
    running_loss, running_acc = 0.0, 0.0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            running_loss += loss.item() * images.size(0)
            running_acc += binary_accuracy(outputs, labels) * images.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_acc / len(dataloader.dataset)
    return epoch_loss, epoch_acc

# تعداد epoch
n_epochs = 20

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(n_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, loss_fn)
    val_loss, val_acc = validate(model, test_loader, loss_fn)

    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f"Epoch {epoch+1}/{n_epochs} | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")


Epoch 1/20 | Train Loss: 0.5251 | Train Acc: 0.7460 | Val Loss: 0.7575 | Val Acc: 0.6132


Epoch 2/20 | Train Loss: 0.4710 | Train Acc: 0.7870 | Val Loss: 0.5611 | Val Acc: 0.7248


Epoch 3/20 | Train Loss: 0.4434 | Train Acc: 0.8005 | Val Loss: 0.5729 | Val Acc: 0.7460


Epoch 4/20 | Train Loss: 0.3969 | Train Acc: 0.8210 | Val Loss: 0.8276 | Val Acc: 0.6231


Epoch 5/20 | Train Loss: 0.3635 | Train Acc: 0.8435 | Val Loss: 0.4517 | Val Acc: 0.7883


...

Epoch 16/20 | Train Loss: 0.0584 | Train Acc: 0.9780 | Val Loss: 1.3491 | Val Acc: 0.6719


Epoch 17/20 | Train Loss: 0.0783 | Train Acc: 0.9735 | Val Loss: 1.0070 | Val Acc: 0.7514


Epoch 18/20 | Train Loss: 0.0545 | Train Acc: 0.9825 | Val Loss: 0.8763 | Val Acc: 0.7653


Epoch 19/20 | Train Loss: 0.0191 | Train Acc: 0.9945 | Val Loss: 1.0435 | Val Acc: 0.7401


Epoch 20/20 | Train Loss: 0.0326 | Train Acc: 0.9915 | Val Loss: 1.1054 | Val Acc: 0.7301
